# 4장 - 좋은 훈련 데이터셋 만들기 - 데이터 전처리

## 4.1 누락된 데이터 다루기

In [3]:
import pandas as pd

#stringIO 클래스: 문자열을 파일처럼 다룰 수 있게 해주는 클래스
#CSV 파일을 만들지 않고 CSV 형태의 문자열을 만든 후 마치 파일에서 읽는 것처럼 동작
from io import StringIO
import sys

#열 제목: A, B, C, D
#총 3개의 행
#행: 샘플, 열: 특성
#누락된 값: 2행 3열, 3행 4열
csv_data = \
'''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''

# 파이썬 2.7을 사용하는 경우
# 다음과 같이 문자열을 유니코드로 변환해야 합니다:
if (sys.version_info < (3, 0)):
    csv_data = unicode(csv_data)

#read_csv 함수: csv파일로부터 데이터를 읽어 반환
df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


pandas에서는 비어 있는 값을 'NaN(Not a Number)'으로 출력<br>
pandas의 데이터 프레임은 왼쪽에 행의 인덱스를 자동으로 출력

In [4]:
#axis: 기본값으로 되어 있음
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

isnull() 함수: df의 항목이 NaN이면 True, 아니면 False 반환<br>
isnull().sum(): True의 개수를 세는 효과(NaN의 개수를 알 수 있음)<br>
axis 기본값: 0 -> 각 열의 NaN의 개수를 헤아리는 효과<br>
axis=1 일 경우: 각 행의 NaN이 개수를 헤아리는 효과

isna(): isnull()과 동일한 메소드

In [5]:
# `values` 속성으로 넘파이 배열을 얻을 수 있습니다
df.values

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6., nan,  8.],
       [10., 11., 12., nan]])

## 4.1.2 누락된 값이 있는 훈련 샘플이나 특성 제외

In [6]:
# 누락된 값이 있는 행을 삭제합니다
df.dropna(axis=0)

,A,B,C,D
0,1.0,2.0,3.0,4.0


2번째 3번째 행 삭제

In [7]:
# 누락된 값이 있는 열을 삭제합니다
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


c열, d열 삭제

In [8]:
# 모든 열이 NaN인 행을 삭제합니다
df.dropna(how='all')  

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


모든 열이 NaN인 것이 없으므로 삭제된 행은 없음<br>
기본값: how='any'

In [9]:
# NaN 아닌 값이 네 개보다 작은 행을 삭제합니다
df.dropna(thresh=4)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [10]:
# 특정 열에 NaN이 있는 행만 삭제합니다(여기서는 'C'열)
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


subset의 매개변수: 리스트로 여러 개의 열을 지정 가능

pandas 데이터 프레임을 사용할 경우 dropna를 사용하면 누락된 값이 있는 행이나 열을 편리하게 삭제 가능

## 4.1.3 누락된 값 대체

In [11]:
# 원래 배열
df.values

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6., nan,  8.],
       [10., 11., 12., nan]])

In [12]:
# 행의 평균으로 누락된 값 대체하기(가장 많이 사용하는 방법)
# 사이킷런에는 누락된 값을 채워주는 클래스 제공
# 누락된 값을 채워주는 클래스들은 대게 sklearn의 impute 모듈 밑에 들어있음
# 가장 대표적인 클래스: SimpleImputer
from sklearn.impute import SimpleImputer
import numpy as np

# SimpleImputer 클래스를 초기화 할 때 missing_values 매개변수 지정 가능
# missing_values 매개변수: 누락된 값이 어떤 값으로 들어가 있는지 지정
# missing_values의 기본값: np.nan
# strategy 매개변수: 누락된 값을 채울 전략 지정
imr = SimpleImputer(missing_values=np.nan, strategy='mean')

# fit 메소드를 사용하지만 실제로 모델을 훈련하는 것은 아님
# fit 메소드를 사용하는 이유: 각 열의 평균값 계산 후 transform 메소드를 이용해 빈 값을 채우는 역할
# 특성 데이터만 변환하는 것이므로 타깃값을 넣지 않고 특성값만 넣어 변환
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

(2,3): 3과 12의 평균값인 7.5로 채워짐 <br>
(3,4): 4와 8의 평균값인 6으로 채워짐

SimpleImputer 클래스: 데이터 변환 클래스 -> 변환기(Transformer) at sklearn

In [13]:
# 특정 함수를 변환기처럼 사용하여 각 샘플에 적용해주는 클래스:  FunctionTransformer(preprocessing 모듈 밑에 들어있음)

from sklearn.preprocessing import FunctionTransformer

# lambda 함수: 
ftr_imr = FunctionTransformer(lambda X: imr.fit_transform(X.T).T)
imputed_data = ftr_imr.fit_transform(df.values)
imputed_data

array([[ 1.        ,  2.        ,  3.        ,  4.        ],
       [ 5.        ,  6.        ,  6.33333333,  8.        ],
       [10.        , 11.        , 12.        , 11.        ]])

fit_transform 메소드: fit과 transform 메소드를 한 번에 제공<br>
5, 6, 8 의 평균인 6.33333잉 계산됨

strategy 매개변수: mean, median(중앙값), most_frequent, constant(임의의 값 지정 가능)<br>
<br>
constant를 사용하면 fill_value 값을 상수로 지정하여 사용 가능

SimpleImputer 클래스의 add_indicator 매개변수를 True로 지정하면 indicator_ 속성이 추가되고 transform() 메서드가 누락된 값의 위치를 포함된 배열을 반환합니다.



In [14]:
# add_indicator 기본값: False
# strategy 기본값: mean

imr = SimpleImputer(add_indicator=True)
imputed_data = imr.fit_transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ,  0. ,  0. ],
       [ 5. ,  6. ,  7.5,  8. ,  1. ,  0. ],
       [10. , 11. , 12. ,  6. ,  0. ,  1. ]])

뒤에 추가된 두 개의 열<br>
누락된 값이 있는 두 개의 열에 대한 인덱스<br>
누락값이 있는 열의 누락된 요소는 1로 표시<br>
: indicator 속성의 추가로 인해 누락값의 위치를 표시

In [15]:
# 추가된 indicator_ 속성은 MissingIndicator 클래스의 객체

imr.indicator_

MissingIndicator(error_on_new=False)

In [16]:
# MissingIndicator 객체의 features_ 속성: 누락된 값이 있는 특성의 인덱스 포함

imr.indicator_.features_

array([2, 3], dtype=int64)

: 3변째 열과 4번째 열을 의미 

In [17]:
imr.indicator_.fit_transform(df.values)

array([[False, False],
       [ True, False],
       [False,  True]])

MissingIndicator 객체의 fit_transform() 메서드: features_ 속성에 담긴 특성에서 누락된 값의 위치를 나타내는 배열을 반환<br>
<br>
세 번째, 네 번째 특성의 누락된 값의 위치<br>
이 배열의 값이 SimpleImputer 객체의 transfrom() 메서드가 반환한 배열의 마지막 두 열에 해당
<br><br>
누락된 값이 있는 곳만 True

IterativeImputer 클래스: 다른 특성을 사용하여 누락된 값 예측<br>
 * SimpleImputer 보다 의미 있는 값으로 대체 가능<br>
 * inital_strategy 매개변수: 누락된 값 초기화
 * mean, median, most_frequent, constant 등<br><br>
예측할 특성을 선택하는 순서
 1. ascending(누락된 값이 적은 특성부터, 기본값)
 2. descending(누락된 값이 큰 특성부터)
 3. roman(왼쪽에서 오른쪽)
 4. arabic(오른쪽에서 왼쪽)
 5. random

1. IterativeImputer 클래스
 * 누락된 값이 있는 것을 타깃처럼 생각하고 다른 특성을 이용하여 예측
 * 특성 예측은 종료 조건을 만족할 때까지 반복
      1. 이전 단계와 절댓값 차이 중 가장 큰 값이 누락된 값을 제외하고 가장 큰 절댓값에 tol 매개변수를 곱한 것보다 작을 경우 종료
       * tol 매개변수 기본값은 1e-3
      2. max_iter 매개변수에서 지정한 횟수에 도달할 때 종료
       * max_iter의 기본값은 10

2. estimator 매개변수: 예측하는 모델 지정
 * 기본적으로 BayesianRidge 클래스 사용
 * n_nearest_features: 예측에 사용할 특성 개수 지정
 * 상관 계수가 높은 특성을 우선하여 랜덤하게 선택
 * 기본값: None(모든 특성을 사용)

In [18]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# 클래스로 객체를 만들고, fit_transform 메소드를 사용하여 호출
iimr = IterativeImputer()
iimr.fit_transform(df.values)

array([[ 1.        ,  2.        ,  3.        ,  4.        ],
       [ 5.        ,  6.        ,  7.00047063,  8.        ],
       [10.        , 11.        , 12.        , 12.99964527]])

3. KNNImputer 클래스: K-최근접 이웃 방법을 사용해 누락된 값을 채움
 * n_neighbors 매개변수: 최근접 이웃의 개수를 지정
 * 기본값: 5
 * 샘플 개수가 n_neighbors 보다 작으면 SimpleImputer(strategy='mean')과 동일한 결과
 * 주변 샘플의 특성으로 보고 특성값의 평균을 냄

In [19]:
from sklearn.impute import KNNImputer

kimr = KNNImputer()
kimr.fit_transform(df.values)

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

In [20]:
# 데이터 프레임의 경우 fillna 함수를 사용하여 빈칸 채우기 가능
#fillna(df.mean()): 빈칸을 평균으로 채우겠다는 의미

df.fillna(df.mean())

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.5,8.0
2,10.0,11.0,12.0,6.0


1. fillna() 메서드의 method 매개변수: 누락된 값채우기 가능
 * bfill 또는 backfill: 누락된 값을 다음 행의 값으로 채움 
 * ffill 또는 pad: 누락된 값을 이전 행의 값으로 채움

In [21]:
df.fillna(method='bfill') # method='backfill'와 같습니다

# 2행d열은 다음행이 없어 값을 채우지 못하고 NaN으로 남아있음

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,12.0,8.0
2,10.0,11.0,12.0,NaN


In [22]:
df.fillna(method='ffill') # method='pad'와 같습니다

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,3.0,8.0
2,10.0,11.0,12.0,8.0


In [23]:
# axis를 1로 설정하였으므로 열을 사용
df.fillna(method='ffill', axis=1)

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,6.0,8.0
2,10.0,11.0,12.0,12.0


## 4.1.4 사이킷런 추정기 API 익히기

사이킷런
 * 변환기: 데이터를 변환하는 클래스(ex.SimpleImputer)
 * 모델
 * 클래스

1. 추정기 클래스
 * fit 메소드에 훈련 데이터와 타깃 데이터를 모두 전달
 * predict 메소드를 통해 test_data의 레이블을 예측
 * score 메소드를 통해 성능 점수 추출

2. 변환기 클래스
 * fit 메소드: train_data에만 적용 -> 훈련 데이터에서 변환에 필요한 데이터를 추출하거나 계산
 * transform 메소드: 훈련 데이터 변환
 * test_data: transform 메소드만 호출하여 train_data를 변환했던 방식으로 test_data를 변환

# 4.2 범주형 데이터 다루기

## 4.2.1 판다스를 사용한 범주형 데이터 인코딩

In [2]:
import pandas as pd

#사이킷런이나 텐서플로와 같은 머신러닝 라이브러리는 숫자로의 변환이 필요

df = pd.DataFrame([['green', 'M', 10.1, 'class2'],
                   ['red', 'L', 13.5, 'class1'],
                   ['blue', 'XL', 15.3, 'class2']])

df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


1. green, red, blue : 순서가 없음
2. 10.1, 13.5, 15.3 : 수치형 데이터
3. class2, class1 : 타깃 데이터
4. M, L, XL : 범주형 데이터지만 순서가 있음

## 4.2.2 순서가 있는 특성 매핑

In [3]:
# dictionary와 map 메소드 사용
size_mapping = {'XL': 3,
                'L': 2,
                'M': 1}

# size 열만 변환
df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


In [4]:
# 1, 2, 3 을 다시 M, L, XL 로 바꾸기

inv_size_mapping = {v: k for k, v in size_mapping.items()}
df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

## 4.2.3 클래스 레이블 인코딩

In [5]:
import numpy as np

# 클래스 레이블을 문자열에서 정수로 바꾸기 위해
# 매핑 딕셔너리를 만듭니다
# enumerate 함수: 주어진 입력을 컬렉션 또는 튜플로 가져와 열거 객체로 반환
class_mapping = {label: idx for idx, label in enumerate(np.unique(df['classlabel']))}
class_mapping

{'class1': 0, 'class2': 1}

In [6]:
# 클래스 레이블을 문자열에서 정수로 바꿉니다
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,1
1,red,2,13.5,0
2,blue,3,15.3,1


In [7]:
# 클래스 레이블을 거꾸로 매핑합니다
inv_class_mapping = {v: k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


In [8]:
from sklearn.preprocessing import LabelEncoder

# 사이킷런의 LabelEncoder을 사용한 레이블 인코딩
class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y

array([1, 0, 1])

In [9]:
# 거꾸로 매핑
class_le.inverse_transform(y)

array(['class2', 'class1', 'class2'], dtype=object)

In [10]:
# class1 과 class2 가 어떤 정수 값으로 매핑 되었는지 확인: classes_ 함수
class_le.classes_

array(['class1', 'class2'], dtype=object)

실제로는 레이블 클래스들이 알아서 레이블 인코더 객체를 만들어 자동으로 변환 수행